## Treatment

In [1]:
import pandas as pd
data_ = pd.read_excel('data/ml.xlsx', sheet_name=None)

key_mapper = {
    'FEMA - Major Disaster - Fire': 'fire',
    'FEMA - Major Disaster - Earthqu': 'earthquake',
    'FEMA - Major Disaster - Hurrica': 'hurricane',
    'FEMA - Major Disaster - Tornado': 'tornado',
    'FEMA - Major Disaster - Flood': 'flood',
    }
data_ = {key_mapper.get(old_key, old_key): value for old_key, value in data_.items()}

Will update the df when more Marie-Laure will fullfill the excel sheet

In [2]:
df_ = data_['fire'].loc[:,['State', 'Declaration Date', 'Incident Type']]

from countries_code import code
df_['State'] = df_['State'].map(code)
df_.head()

,State,Declaration Date,Incident Type
0,CA,2021-09-12,Fire
1,CA,2021-08-24,Fire
2,WA,2021-02-04,Fire
3,CO,2021-01-15,Fire
4,MT,2021-09-30,Fire


In [5]:
import plotly.express as px

fig = px.choropleth(
    df_.groupby(["State"]).count().reset_index(),
    locations="State",
    color="Declaration Date",
    color_continuous_scale="spectral_r",
    hover_name="State",
    locationmode="USA-states",
    scope="usa",
    labels={"Declaration Date": "Number of incidents"},
)
fig.add_scattergeo(
    locations=df_["State"], locationmode="USA-states", text=df_["State"], mode="text"
)

fig.show()

## Outcomes


Files need to be merged

In [6]:
files_ = [
    f"data/CDC/{data_file}"
    for data_file in os.listdir("data/CDC")
    if "txt" in data_file
]

columns = ["State", "Month Code", "UCD - ICD Chapter", "Deaths"]

df_outcomes_ = pd.concat(
    objs=[
        pd.read_csv(file, delimiter="\t", parse_dates=["Month Code"])[columns].dropna()
        for file in files_
    ]
)

from countries_code import code

df_outcomes_["State"] = df_outcomes_["State"].map(code)

df_outcomes_.rename(
    columns={
        "State": "state",
        "UCD - ICD Chapter": "cause",
        "Deaths": "deaths",
        "Month Code": "time",
    },
    inplace=True,
)

df_outcomes = df_outcomes_.set_index(["state", "time", "cause"])
df_outcomes

deaths
state time       cause                                                     
AL    2010-01-01 Certain infectious and parasitic diseases            131.0
                 Neoplasms                                            889.0
                 Diseases of the blood and blood-forming organs ...    17.0
                 Endocrine, nutritional and metabolic diseases        170.0
                 Mental and behavioural disorders                     261.0
...                                                                     ...
WY    2017-12-01 Diseases of the circulatory system                   121.0
                 Diseases of the respiratory system                    59.0
                 Diseases of the digestive system                      21.0
                 Diseases of the genitourinary system                  12.0
                 External causes of morbidity and mortality            35.0

[90532 rows x 1 columns]

In [9]:
import plotly
import numpy as np

date = df_outcomes_.time.min()
data_slider = []
step_ = 3

zmin, zmax = df_outcomes_.deaths.min(), df_outcomes_.deaths.max()


for add_month in range(
    1,
    int(
        (df_outcomes_.time.max() - df_outcomes_.time.min()) / np.timedelta64(1, "M") + 1
    ),
    step_,
):
    df_segmented = df_outcomes_[
        (
            df_outcomes_["time"]
            == df_outcomes_.time.min() + pd.DateOffset(months=add_month)
        )
    ]

    data_mth = dict(
        type="choropleth",
        locations=df_segmented["state"],
        z=df_segmented["deaths"],
        locationmode="USA-states",
        zmin=zmin,
        zmax=zmax/8,
        colorbar={"title": "# Deaths"},
        colorscale="Viridis",
    )

    data_slider.append(data_mth)

steps = []
for i in range(len(data_slider)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(data_slider)],
        label=str((df_outcomes_.time.min() + pd.DateOffset(months=i *step_)))[:7],
    )
    step["args"][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]

layout = dict(
    title="Death By State Since 2010",
    geo=dict(scope="usa", projection={"type": "albers usa"}),
    sliders=sliders,
)

fig = dict(data=data_slider, layout=layout)
plotly.offline.iplot(fig)

In [85]:
df_outcomes_.cause.unique()

array(['Certain infectious and parasitic diseases', 'Neoplasms',
       'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism',
       'Endocrine, nutritional and metabolic diseases',
       'Mental and behavioural disorders',
       'Diseases of the nervous system',
       'Diseases of the circulatory system',
       'Diseases of the respiratory system',
       'Diseases of the digestive system',
       'Diseases of the musculoskeletal system and connective tissue',
       'Diseases of the genitourinary system',
       'Certain conditions originating in the perinatal period',
       'Congenital malformations, deformations and chromosomal abnormalities',
       'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified',
       'External causes of morbidity and mortality',
       'Diseases of the skin and subcutaneous tissue',
       'Pregnancy, childbirth and the puerperium',
       'Codes for special purpose

## Cofeatures

Vera Institute gives data on incarceration in the US

In [10]:
df = pd.DataFrame()

### Vera Institute DataSet

In [360]:
df_vera = pd.read_csv('data/veraInstitute/incarceration_trends.csv')

"lets reindex to sum over states"
df_vera = df_vera.set_index(['year','state', 'county_name'])
df_vera.loc[(1970, slice(None), slice(None)),:]

yfips   fips  total_pop  total_pop_15to64  \
year state county_name                                                        
1970 AL    Autauga County     197001001   1001      24661             14154   
           Baldwin County     197001003   1003      59531             35043   
           Barbour County     197001005   1005      22644             13210   
           Bibb County        197001007   1007      13824              7982   
           Blount County      197001009   1009      27042             16652   
...                                 ...    ...        ...               ...   
     WY    Sweetwater County  197056037  56037      18566             11240   
           Teton County       197056039  56039       4897              3139   
           Uinta County       197056041  56041       7083              4223   
           Washakie County    197056043  56043       7560              4602   
           Weston County      197056045  56045       6270              3690   

                              female_pop_15to64  male_pop_15to64  \
year state county_name                                             
1970 AL    Autauga County                  7293             6861   
           Baldwin County                 18072            16971   
           Barbour County                  7151             6059   
           Bibb County                     4099             3883   
           Blount County                   8538             8114   
...                                         ...              ...   
     WY    Sweetwater County               5598             5642   
           Teton County                    1569             1570   
           Uinta County                    2082             2141   
           Washakie County                 2248             2354   
           Weston County                   1868             1822   

                              aapi_pop_15to64  black_pop_15to64  \
year state county_name                                            
1970 AL    Autauga County                 NaN               NaN   
           Baldwin County                 NaN               NaN   
           Barbour County                 NaN               NaN   
           Bibb County                    NaN               NaN   
           Blount County                  NaN               NaN   
...                                       ...               ...   
     WY    Sweetwater County              NaN               NaN   
           Teton County                   NaN               NaN   
           Uinta County                   NaN               NaN   
           Washakie County                NaN               NaN   
           Weston County                  NaN               NaN   

                              latinx_pop_15to64  native_pop_15to64  ...  \
year state county_name                                              ...   
1970 AL    Autauga County                   NaN                NaN  ...   
           Baldwin County                   NaN                NaN  ...   
           Barbour County                   NaN                NaN  ...   
           Bibb County                      NaN                NaN  ...   
           Blount County                    NaN                NaN  ...   
...                                         ...                ...  ...   
     WY    Sweetwater County                NaN                NaN  ...   
           Teton County                     NaN                NaN  ...   
           Uinta County                     NaN                NaN  ...   
           Washakie County                  NaN                NaN  ...   
           Weston County                    NaN                NaN  ...   

                              native_prison_pop_rate white_prison_pop_rate  \
year state county_name                                                       
1970 AL    Autauga County                        NaN                   NaN   
           Baldwin County              

In [365]:
# use multi index capabilities to merge by state
df_vera = df_vera.groupby(level=[0, 1]).sum()
df_vera = pd.concat(
    [
        (
            df_vera.loc[:, ["total_jail_pop", "total_prison_pop"]].sum(axis=1)
            / df_vera.loc[:, "total_pop"]
        ).rename("imprisonment_rate"),
        df_vera["total_pop"],
    ],
    axis=1,
)

df_vera

imprisonment_rate  total_pop
year state                              
1970 AK              0.000392     255117
     AL              0.000874    3454557
     AR              0.000633    1932816
     AZ              0.001190    1799531
     CA              0.001380   20045806
...                       ...        ...
2018 VT              0.000000     626299
     WA              0.001694    7535591
     WI              0.002294    5813568
     WV              0.002932    1805832
     WY              0.002666     577737

[2499 rows x 2 columns]

## Census Datasets

Data coming from https://www.census.gov/acs/www/data/data-tables-and-tools/data-profiles/2020/

In [ ]:
def yearly_to_monthly(df: pd.DataFrame) -> pd.DataFrame:
    """The dataframe needs to cointain the state and year columns."""
    df_ = pd.concat(
        {month: df for month in [1, 12]}, names=["month"]
    ).reorder_levels(["year", "month", "state"]).reset_index().set_index("state")

    df_["date"] = (
        df_["year"].astype(str) + "-" + df_["month"].astype(str)
    ).astype("datetime64[ns]")

    return df_.reset_index().set_index(['state', 'date']).drop(columns=["year", "month"])

In [346]:
from countries_code import code


def census_loader(path: str = "data/census/economics", pattern: str = "ACSDP1Y"):
    df_ = {
        year: processed_dataframe
        for year, processed_dataframe in zip(
            map(
                # extract the year from the name of the file
                lambda l: l[
                    l.index(pattern)
                    + len(pattern) : l.index(pattern)
                    + len(pattern)
                    + 4
                ],
                files_,
            ),
            map(
                # process the dataframe: map state names, remove uncertaincy and transpose
                lambda df: df.rename(columns={"Unnamed: 0": ""})
                # .set_index("")
                .iloc[
                    1:,
                    ~df.columns.str.contains("^Unnamed")
                    # 1 slice because unamed 0 became index already
                ]
                .replace("(X)", np.nan)
                .rename(columns=code)
                .transpose()
                .reset_index()
                .rename(columns={"index": "state"}),
                [pd.read_csv(file, header=[0]) for file in files_],
            ),
        )
    }

    for year, df in df_.items():
        df["year"] = year
        df.set_index(["state", "year"], inplace=True)
        # df.columns = df.columns.str.strip()
        # df.set_index(["state", "year"], inplace=True)

    return pd.concat(df_.values())

### Economics

In [355]:
df_economics=census_loader(path="data/census/economics")
df_economics = df_economics.rename(columns={i: "e{}".format(i) for i in range(len(df_economics.columns))})
df_economics

,,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,...,e136,e137,e138,e139,e140,e141,e142,e143,e144,145
state,year,,,,,,,,,,,,,,,,,,,,,
AL,2011,NaN,"3,808,975","2,255,995","2,240,441","1,981,095","259,346","15,554","1,552,980","2,240,441",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,2011,NaN,"554,678","394,406","377,710","344,343","33,367","16,696","160,272","377,710",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZ,2011,NaN,"5,044,259","3,040,762","3,025,208","2,687,991","337,217","15,554","2,003,497","3,025,208",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR,2011,NaN,"2,301,244","1,368,632","1,363,235","1,235,755","127,480","5,397","932,612","1,363,235",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,2011,NaN,"29,530,797","18,869,522","18,737,749","16,426,694","2,311,055","131,773","10,661,275","18,737,749",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WA,2015,NaN,"5,741,873","3,658,933","3,613,427","3,397,974","215,453","45,506","2,082,940","3,613,427",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WV,2015,NaN,"1,508,224","797,526","796,708","738,438","58,270",818,"710,698","796,708",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WI,2015,NaN,"4,627,790","3,094,071","3,091,421","2,959,655","131,766","2,650","1,533,719","3,091,421",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Demographic

In [356]:
df_demographic = census_loader(path="data/census/demographic")
df_demographic = df_demographic.rename(columns={i: "d{}".format(i) for i in range(len(df_demographic.columns))})
df_demographic


,,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,...,d136,d137,d138,d139,d140,d141,d142,d143,d144,145
state,year,,,,,,,,,,,,,,,,,,,,,
AL,2011,NaN,"3,808,975","2,255,995","2,240,441","1,981,095","259,346","15,554","1,552,980","2,240,441",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,2011,NaN,"554,678","394,406","377,710","344,343","33,367","16,696","160,272","377,710",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZ,2011,NaN,"5,044,259","3,040,762","3,025,208","2,687,991","337,217","15,554","2,003,497","3,025,208",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR,2011,NaN,"2,301,244","1,368,632","1,363,235","1,235,755","127,480","5,397","932,612","1,363,235",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,2011,NaN,"29,530,797","18,869,522","18,737,749","16,426,694","2,311,055","131,773","10,661,275","18,737,749",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WA,2015,NaN,"5,741,873","3,658,933","3,613,427","3,397,974","215,453","45,506","2,082,940","3,613,427",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WV,2015,NaN,"1,508,224","797,526","796,708","738,438","58,270",818,"710,698","796,708",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WI,2015,NaN,"4,627,790","3,094,071","3,091,421","2,959,655","131,766","2,650","1,533,719","3,091,421",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Housing